# SQL Subqueries - Lab done by `Eugene Maina`

## Introduction

Now that you've seen how subqueries work, it's time to get some practice writing them! Not all of the queries will require subqueries, but all will be a bit more complex and require some thought and review about aggregates, grouping, ordering, filtering, joins and subqueries. Good luck!  

## Objectives

You will be able to:

* Write subqueries to decompose complex queries

## CRM Database ERD

Once again, here's the schema for the CRM database you'll continue to practice with.

<img src="https://curriculum-content.s3.amazonaws.com/data-science/images/Database-Schema.png" width="600">

## Connect to the Database

As usual, start by importing the necessary packages and connecting to the database `data.sqlite`.

In [2]:
# Your code here; import the necessary packages
import pandas as pd
import sqlite3

In [3]:
# Your code here; create the connection
conn = sqlite3.connect('data.sqlite')
cur = conn.cursor()

## Write an Equivalent Query using a Subquery

The following query works using a `JOIN`. Rewrite it so that it uses a subquery instead.

```
SELECT
    customerNumber,
    contactLastName,
    contactFirstName
FROM customers
JOIN orders 
    USING(customerNumber)
WHERE orderDate = '2003-01-31'
;
```

In [4]:
# Your code here
orders = '''SELECT customerNumber, contactLastName, contactFirstName
            FROM customers
            WHERE customerNumber IN (SELECT customerNumber
                                      FROM orders
                                      WHERE orderDate = '2003-01-31');'''
pd.read_sql_query(orders, conn)

,customerNumber,contactLastName,contactFirstName
0,141,Freyre,Diego


## Select the Total Number of Orders for Each Product Name

Sort the results by the total number of items sold for that product.

In [5]:
# Your code here
orders_by_product = '''
                    SELECT p.productName, COUNT(od.orderNumber) AS num_orders
                    FROM products p
                    JOIN orderdetails AS od
                    ON p.productCode = od.productCode
                    GROUP BY p.productName
                    ORDER BY num_orders DESC
;'''

pd.read_sql_query(orders_by_product, conn, index_col='productName')

,num_orders
productName,
1992 Ferrari 360 Spider red,53
P-51-D Mustang,28
HMS Bounty,28
F/A 18 Hornet 1/72,28
Diamond T620 Semi-Skirted Tanker,28
...,...
1932 Alfa Romeo 8C2300 Spider Sport,25
1917 Grand Touring Sedan,25
1911 Ford Town Car,25


## Select the Product Name and the  Total Number of People Who Have Ordered Each Product

Sort the results in descending order.

### A quick note on the SQL  `SELECT DISTINCT` statement:

The `SELECT DISTINCT` statement is used to return only distinct values in the specified column. In other words, it removes the duplicate values in the column from the result set.

Inside a table, a column often contains many duplicate values; and sometimes you only want to list the unique values. If you apply the `DISTINCT` clause to a column that has `NULL`, the `DISTINCT` clause will keep only one NULL and eliminates the other. In other words, the DISTINCT clause treats all `NULL` “values” as the same value.

In [6]:
# Your code here
# Hint: because one of the tables we'll be joining has duplicate customer numbers, you should use DISTINCT

orders_by_customer = '''
SELECT p.productName,  COUNT(DISTINCT ord.customerNumber) AS num_of_customers
        FROM products AS p
        JOIN orderdetails AS o
        ON p.productCode = o.productCode
        JOIN orders AS ord
        ON o.orderNumber = ord.orderNumber
        GROUP BY p.productName
        ORDER BY num_of_customers DESC;'''
pd.read_sql_query(orders_by_customer, conn, index_col='productName')

,num_of_customers
productName,
1992 Ferrari 360 Spider red,40
Boeing X-32A JSF,27
1972 Alfa Romeo GTA,27
1952 Alpine Renault 1300,27
1934 Ford V8 Coupe,27
...,...
1958 Chevy Corvette Limited Edition,19
2002 Chevy Corvette,18
1969 Chevrolet Camaro Z28,18


## Select the Employee Number, First Name, Last Name, City (of the office), and Office Code of the Employees Who Sold Products That Have Been Ordered by Fewer Than 20 people.

This problem is a bit tougher. To start, think about how you might break the problem up. Be sure that your results only list each employee once.

In [12]:
# Your code here

employees_with_under_20_product_sales =''' SELECT DISTINCT e.employeeNumber, e.firstName || ' ' ||  e.lastName AS "Full Name", o.city, o.officeCode
                                                FROM employees AS e
                                                JOIN offices AS o 
                                                ON e.officeCode = o.officeCode
                                                JOIN customers AS c 
                                                ON e.employeeNumber = c.salesRepEmployeeNumber
                                                JOIN orders AS od 
                                                ON c.customerNumber = od.customerNumber
                                                JOIN orderdetails AS odd
                                                ON od.orderNumber = odd.orderNumber
                                                WHERE odd.productCode IN (
                                                        SELECT productCode
                                                        FROM orderdetails
                                                        GROUP BY productCode
                                                        HAVING COUNT(DISTINCT productCode) < 20
                                                    )
;'''

pd.read_sql_query(employees_with_under_20_product_sales, conn, index_col='employeeNumber')

,Full Name,city,officeCode
employeeNumber,,,
1216,Steve Patterson,Boston,2
1504,Barry Jones,London,7
1286,Foon Yue Tseng,NYC,3
1370,Gerard Hernandez,Paris,4
1401,Pamela Castillo,Paris,4
1323,George Vanauf,NYC,3
1621,Mami Nishi,Tokyo,5
1501,Larry Bott,London,7
1165,Leslie Jennings,San Francisco,1


## Select the Employee Number, First Name, Last Name, and Number of Customers for Employees Whose Customers Have an Average Credit Limit Over 15K

In [16]:
# Your code here
over_15K_credit_limit = ''' SELECT DISTINCT e.employeeNumber, e.firstName || ' ' ||  e.lastName AS "Full Name", COUNT(DISTINCT c.customerNumber) AS num_of_customers
                            FROM employees AS e
                            JOIN customers AS c
                            ON e.employeeNumber = c.salesRepEmployeeNumber
                            WHERE c.creditLimit > 15000
                            GROUP BY e.employeeNumber
                            ORDER BY num_of_customers DESC;'''

pd.read_sql_query(over_15K_credit_limit, conn, index_col='employeeNumber')

,Full Name,num_of_customers
employeeNumber,,
1401,Pamela Castillo,10
1504,Barry Jones,9
1501,Larry Bott,8
1323,George Vanauf,8
1370,Gerard Hernandez,7
1337,Loui Bondur,6
1286,Foon Yue Tseng,6
1216,Steve Patterson,6
1188,Julie Firrelli,6


## Summary

In this lesson, you got to practice some more complex SQL queries, some of which required subqueries. There's still plenty more SQL to be had though; hope you've been enjoying some of these puzzles!